In [2]:
import numpy as np
import pandas as pd
import glob 
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import ast
#import tensorflow as tf

In [3]:
metadata = pd.read_csv('metadata.csv')
metadata.head()

,title,date,for,against,id
0,Capitalism Is a Blessing,2019-11-12,"['John Mackey', 'Katherine Mangu-Ward']","['Bhaskar Sunkara', 'Richard D. Wolff']",d20191112
1,Parenting Is Overrated,2019-10-29,"['Robert Plomin', 'Nancy Segal']","['Paige Harden', 'Ann Pleshette Murphy']",d20191029
2,Europe Has Declared War on American Tech Compa...,2019-10-22,"['Roslyn Layton', 'Berin Szóka']","['Marietje Schaake', 'Ramesh Srinivasan']",d20191022
3,Replace Private Insurance with Medicare for All,2019-09-17,"['Dr. Adam Gaffney', 'Joseph Sanberg']","['Nick Gillespie', 'Sally Pipes']",d20190917
4,Unresolved: Shifting Power in the Middle East,2019-09-12,"['Michael Doran', 'Reuel Marc Gerecht', 'Berna...","['Brett McGurk', 'Barbara Slavin']",d20190912


In [4]:
cleaned_files = glob.glob('cleaned_transcript_data/*.csv')

In [5]:
def determine_side(name, sidedict):
    for side in sidedict:
        if name in sidedict[side] or name.upper() in sidedict[side]:
            return side
    return 'unknown'

In [8]:
def for_against_scripts(filepath):
    fid = filepath[filepath.find('20') - 1: filepath.find('csv') - 1]
    metadata.loc[metadata['id'] == fid]
    sidedict = {}
    for side in ['for', 'against']:
        sidedict[side] = metadata.loc[metadata['id'] == fid][side].item().upper()
    cleaned = pd.read_csv(filepath).drop(columns = 'Unnamed: 0')
    cleaned['side'] = cleaned['Speaker'].apply(lambda x: determine_side(x, sidedict))
    if 'unknown' in cleaned['side'].values:
        print(filepath)
        return pd.DataFrame(data = {'side': [], 'script': []})
    new_sides = []
    new_scripts = []
    new_rounds = []
    last_side = 0
    last_round = 0
    for i in range(len(cleaned)):
        side = cleaned['side'][i]
        script = cleaned['Script'][i]
        round_ = cleaned['Round'][i]
        if side == last_side and round_ == last_round :
            new_scripts[-1] = new_scripts[-1] + ' ' + script
        else:
            new_sides.append(side)
            new_scripts.append(script)
            new_rounds.append(round_)
            last_side = side
            last_round = round_
    return pd.DataFrame(data = {'side': new_sides, 'script': new_scripts, 'round': new_rounds})

In [9]:
for filepath in cleaned_files:
    df = for_against_scripts(filepath)
    fid = filepath[filepath.find('20') - 1: filepath.find('csv') - 1]
    df.to_csv('For Against Scripts/for_against_scripts_' + fid + '.csv', index = False)

cleaned_transcript_data/cleaned_transcript_d20100209.csv


In [12]:
cleaned = pd.read_csv('cleaned_transcript_data/cleaned_transcript_d20100209.csv')
sidedict = {}
fid = 'd20100209'
for side in ['for', 'against']:
    sidedict[side] = metadata.loc[metadata['id'] == fid][side].item().upper()
cleaned = pd.read_csv(filepath).drop(columns = 'Unnamed: 0')
cleaned['side'] = cleaned['Speaker'].apply(lambda x: determine_side(x, sidedict))

In [14]:
sidedict

{'for': "['ROGER COHEN', 'RASHAD KHALIDI']",
 'against': "['STUART EIZENSTAT', 'ITAMAR RABINOVICH']"}

In [13]:
cleaned

,Speaker,Script,Round,side
0,Mike McConnell,"Well, a few things, not too many.",NaN,unknown
1,Mike McConnell,"Well, actually I am going to share a little bi...",1.0,unknown
2,Mike McConnell,-- goes with that long history.,1.0,unknown
3,Mike McConnell,The translation --,1.0,unknown
4,Mike McConnell,Thank you. The translation of experience is ag...,1.0,unknown
5,Mike McConnell,That's what age does for you. Let me complimen...,1.0,unknown
6,Mike McConnell,And I would summarize by saying we have laws a...,1.0,unknown
7,Bruce Schneier,"Actually, I think it's an East Coast crypto th...",1.0,unknown
8,Bruce Schneier,"So, we're here today to debate the motion that...",1.0,unknown
9,Bruce Schneier,"This year, London Times, March of this year: ""...",1.0,unknown
